<a href="https://colab.research.google.com/github/AmriteshDOT/nlp_/blob/main/src/essayMLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
essay = pd.read_csv("/data/train.csv")

In [ ]:
df=essay.copy()
# df=df.head(500)

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
sp_token = "[BR]"
modeltype = "distilbert/distilbert-base-uncased"

In [ ]:
def normalize_text(s):
    if pd.isna(s):
        return ""
    return str(s).replace("\r\n", "\n").replace("\r", "\n").replace("\n", f" {sp_token} ")

df['full_text']=df['full_text'].apply(normalize_text)

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_scores, val_scores = train_test_split(
    df['full_text'].to_list(), df['score'].to_list(), test_size=0.1, random_state=42
)

In [ ]:
with open("train_text.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(train_texts))
with open("val_text.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(val_texts))

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(modeltype, use_fast=True)
if sp_token not in tokenizer.get_added_vocab():
    tokenizer.add_special_tokens({"additional_special_tokens": [sp_token]})


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(modeltype)
model.resize_token_embeddings(len(tokenizer))

Embedding(30523, 768, padding_idx=0)

In [ ]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
train_ds = LineByLineTextDataset(tokenizer=tokenizer, file_path="/kaggle/working/train_text.txt", block_size=256)
val_ds = LineByLineTextDataset(tokenizer=tokenizer, file_path="/kaggle/working/val_text.txt", block_size=256)
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/kaggle/working",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    fp16=torch.cuda.is_available(),
    eval_steps=500,
    seed=42
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=train_ds,
    eval_dataset=val_ds,
)

In [ ]:
out_dir = "/models"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
trainer.train()
trainer.save_model(out_dir)
tokenizer.save_pretrained(out_dir)